In [49]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import api_keys
from spotipy.oauth2 import SpotifyOAuth
import os
import requests
from dotenv import load_dotenv
import base64
import json
import webbrowser
import urllib

## https://developer.spotify.com/documentation/web-api/reference/get-users-saved-tracks

In [50]:
load_dotenv()
REDIRECT_URI = "http://localhost"
CLIENT_ID = os.getenv("client_id")
CLIENT_SECRET = os.getenv("client_secret")
SCOPE = 'user-library-read'

## Request User Authorization

In [51]:
# Create the authorization URL
params = {
    'response_type': 'code',
    'client_id': CLIENT_ID,
    'scope': SCOPE,
    'redirect_uri': REDIRECT_URI,
}
url = f"https://accounts.spotify.com/authorize?{urllib.parse.urlencode(params)}"

# Open the authorization URL in the default web browser
webbrowser.open(url)

True

In [52]:
CODE = "AQDjyfYuyZSuZdLjgELihlbLb54-wyK0E11W0s0TbVFWGr66PA9jwv9Om_hxiRLO5MgSDa36oVdxpD0MUO6d8nXaVMmBRwqVr0Ymn2AAkgSDaZ3fbBW1fVdCzMtbP3KCi5lhl3XfJQJJEbUbCONlqY8FyNlVLiUcqQWCi0C4keA2DVWErc6r"

## Get Token Info
- Lasts only 1 hour

In [53]:
def get_token_info():
    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode('utf-8')).decode('utf-8')
    # auth_string = client_id + ":" + client_secret
    # auth_bytes = auth_string.encode("utf-8")
    # auth_base64 = str(base64.b64encode(auth_bytes), "utf-8" )

    url = "https://accounts.spotify.com/api/token"
    headers = {
            "Authorization": f"Basic {auth_header}",
            "Content-Type": "application/x-www-form-urlencoded",
    }
    data = {
            "grant_type": "authorization_code", 
            "code" : CODE, 
            "redirect_uri": REDIRECT_URI}
    result = requests.post(url, headers=headers, data=data)
    json_results = json.loads(result.content)
    token = json_results["access_token"]
    refresh_token = json_results["refresh_token"]
    return json_results

token_info = get_token()

In [54]:
token_info

{'access_token': 'BQDHw30lvQfXNvWXdvG2KNXl-k8guuZ0LjCJ1_0SkdP2hwzeWbnEMqd3zGfexBbq0x7XiMOKcE9QBR57P3jzExitr-t45v0HJ1AeKb0Y3Nl50oVjJGWCC8onJNVMA8a53kv1N8xJ7JNzifQuEYv_0gIzyOuZE4R_1_OLGZKdj-bPbml11aWo4zUnHEikQtM',
 'token_type': 'Bearer',
 'expires_in': 3600,
 'refresh_token': 'AQA4MVKO8kHZbyDAfuedcoa0lE7RYIYSd568x5trIM-G3O9RysTqDg47DItgM1oafO_wE0YvHBi711MSXvqAOrQ7iEAx4oPnxuWkxPLpU9ivlzTCt2COY9i8br30JSHfcaA',
 'scope': 'user-library-read'}

In [55]:
token = token_info["access_token"]
refresh_token = token_info["refresh_token"]
scope = token_info["scope"]

## Getting Refresh Token

In [28]:
# def authorize():
#     # The scopes your application needs access to
#     scopes = ['user-library-read']

#     # Create the authorization URL
#     params = {
#         'client_id': client_id,
#         'response_type': 'code',
#         'redirect_uri': REDIRECT_URI,
#         'scope': ' '.join(scopes)

## Refresh Token

In [29]:

# client_id = os.getenv("client_id")
# client_secret = os.getenv("client_secret")

# def get_token():
#     auth_string = f"{client_id}:{client_secret}"
#     auth_bytes = auth_string.encode("utf-8")
#     auth_base64 = str(base64.b64encode(auth_bytes), "utf-8" )

#     url = "https://accounts.spotify.com/api/token"
#     headers = {
#             "Authorization": f"Basic {auth_base64}",
#             "Content-Type": "application/x-www-form-urlencoded"
#     }
#     data = {"grant_type": "refresh_token"}
#     result = post(url, headers=headers, data=data)
#     json_results = json.loads(result.content)
#     token = json_results["access_token"]
#     refresh_token = json_results["refresh_token"]
#     return token, refresh_token

# refresh_token = get_token()

In [30]:
# refresh_token

## Get Auth Header

In [56]:
def get_auth_header(token):
    return f"Bearer " + token

In [57]:
get_auth_header(token)

'Bearer BQDHw30lvQfXNvWXdvG2KNXl-k8guuZ0LjCJ1_0SkdP2hwzeWbnEMqd3zGfexBbq0x7XiMOKcE9QBR57P3jzExitr-t45v0HJ1AeKb0Y3Nl50oVjJGWCC8onJNVMA8a53kv1N8xJ7JNzifQuEYv_0gIzyOuZE4R_1_OLGZKdj-bPbml11aWo4zUnHEikQtM'

## Post Request
- Request using token for API endpoints

In [58]:
SPOTIFY_GET_CURRENT_USER_SAVED_TRACKS_URL = "https://api.spotify.com/v1/me/tracks?limit=50&offset=0"
SPOTIFY_GET_CURRENT_TRACK_URL = 'https://api.spotify.com/v1/me/player/currently-playing'

In [59]:
def get_current_user_tracks(url):
    response = requests.get(url=SPOTIFY_GET_CURRENT_USER_SAVED_TRACKS_URL, headers={"Authorization" : f"{get_auth_header(token)}"})
    resp_json = response.json()
    return resp_json

In [74]:
def loop_through():
    # increment_of50_list = [i for i in range(0, get_current_user_tracks()["total"], 50)]
    increment_of50_list_test = [i for i in range(0, int(get_current_user_tracks(SPOTIFY_GET_CURRENT_USER_SAVED_TRACKS_URL)["total"]) - 4300, 50)]
    songs = []
    full_list = []
    for i in increment_of50_list_test:
        spotify_50_loop_url = f"https://api.spotify.com/v1/me/tracks?limit=50&offset={i}"
        results = get_current_user_tracks(spotify_50_loop_url)
        full_list.append(results)
        # for song in results:
            # song_name =results["items"][0]["track"]["name"]
            # songs.append(song_name)
    return full_list
            

In [75]:
loop_through()

[{'href': 'https://api.spotify.com/v1/me/tracks?offset=0&limit=50',
  'items': [{'added_at': '2023-06-09T15:26:19Z',
    'track': {'album': {'album_type': 'single',
      'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5eIbEEQnDM8yuDVB0bimSP'},
        'href': 'https://api.spotify.com/v1/artists/5eIbEEQnDM8yuDVB0bimSP',
        'id': '5eIbEEQnDM8yuDVB0bimSP',
        'name': 'RL Grime',
        'type': 'artist',
        'uri': 'spotify:artist:5eIbEEQnDM8yuDVB0bimSP'},
       {'external_urls': {'spotify': 'https://open.spotify.com/artist/12Zk1DFhCbHY6v3xep2ZjI'},
        'href': 'https://api.spotify.com/v1/artists/12Zk1DFhCbHY6v3xep2ZjI',
        'id': '12Zk1DFhCbHY6v3xep2ZjI',
        'name': '070 Shake',
        'type': 'artist',
        'uri': 'spotify:artist:12Zk1DFhCbHY6v3xep2ZjI'}],
      'available_markets': ['AD',
       'AE',
       'AG',
       'AL',
       'AM',
       'AO',
       'AR',
       'AT',
       'AU',
       'AZ',
       'BA',
       'BB

In [82]:
test_results = loop_through()

In [94]:
for i in test_results:    
    total_global = []
    for index, i in enumerate(i["items"]):
        # song_name =i["items"][index]["track"]["name"]
        total = {}
        total[index] = i
        total_global.append(total)
    print(total_global)

[{0: {'added_at': '2023-06-09T15:26:19Z', 'track': {'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5eIbEEQnDM8yuDVB0bimSP'}, 'href': 'https://api.spotify.com/v1/artists/5eIbEEQnDM8yuDVB0bimSP', 'id': '5eIbEEQnDM8yuDVB0bimSP', 'name': 'RL Grime', 'type': 'artist', 'uri': 'spotify:artist:5eIbEEQnDM8yuDVB0bimSP'}, {'external_urls': {'spotify': 'https://open.spotify.com/artist/12Zk1DFhCbHY6v3xep2ZjI'}, 'href': 'https://api.spotify.com/v1/artists/12Zk1DFhCbHY6v3xep2ZjI', 'id': '12Zk1DFhCbHY6v3xep2ZjI', 'name': '070 Shake', 'type': 'artist', 'uri': 'spotify:artist:12Zk1DFhCbHY6v3xep2ZjI'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CD', 'CG', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'ET', 'FI', 'FJ', 'FM', 

In [39]:
get_current_user_tracks()["total"]

4489

In [ ]:
increment_of50_list = [i for i in range(0, 5000, 50)]
increment_of50_list_test = [i for i in range(0, 300, 50)]
data = {}

while results["next"] is not None:
    for i in increment_of50_list:
        results = sp.current_user_saved_tracks(limit=50, offset=i)
        # added_at_to_my_playlist: results_temp["items"][0]["added_at"]
        # artist_name: results_temp["items"][0]["track"]["album"]["artists"][0]["name"]
        # unique_artist_id : results_temp["items"][0]["track"]["album"]["artists"][0]["uri"]
        # release_date: results_temp["items"][0]["track"]["album"]["release_date"]
        # song_name : results_temp["items"][0]["track"]["name"]
        # song_lengh_seconds : int(results_temp["items"][0]["track"]["duration_ms"])/1000
        # song_explicit : results_temp["items"][0]["track"]["explicit"]
        # song_linkId : results_temp["items"][0]["track"]["external_urls"]["spotify"]
        # song_popularity : results_temp["items"][0]["track"]["popularity"]
        # album_name : results_temp["items"][1]["track"]["album"]["name"]
        # spotify_uri_track : results_temp["items"][0]["track"]["album"]["uri"]


In [136]:
### Items to add:
# added_at_to_my_playlist: results_temp["items"][0]["added_at"]
# artist_name: results_temp["items"][0]["track"]["album"]["artists"][0]["name"]
# unique_artist_id : results_temp["items"][0]["track"]["album"]["artists"][0]["uri"]
# release_date: results_temp["items"][0]["track"]["album"]["release_date"]
# song_name : results_temp["items"][0]["track"]["name"]
# song_lengh_seconds : int(results_temp["items"][0]["track"]["duration_ms"])/1000
# song_explicit : results_temp["items"][0]["track"]["explicit"]
# song_linkId : results_temp["items"][0]["track"]["external_urls"]["spotify"]
# song_popularity : results_temp["items"][0]["track"]["popularity"]
# album_name : results_temp["items"][1]["track"]["album"]["name"]
# spotify_uri_track : results_temp["items"][0]["track"]["album"]["uri"]

'2023-06-02T02:50:17Z'

In [93]:
# results_temp["items"]["release_date"]
for i in results_temp["items"][0]:
    print(i)

added_at
track


In [67]:
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

0 Wiley Beckett  –  Disco Nights
1 Ozer  –  Polo Fendi
2 Young Franco  –  Juice
3 Jungle  –  Romeo
4 Kota the Friend  –  Red Lights (feat. Hello O'shay)
5 3LAU  –  Better With You - Kastra & twoDB Remix
6 Boombox Cartel  –  All I Want
7 Man Cub  –  Good Enough
8 Morgan Wallen  –  You Proof
9 Tory Lanez  –  ‘87 Stingray
10 Tory Lanez  –  Hurt From Mercury
11 Richy Mitch & The Coal Miners  –  Sage
12 Masego  –  What You Wanna Try
13 Marshmello  –  Fell In Love
14 Paul Russell  –  Ms. Poli Sci
15 NLE Choppa  –  DO IT AGAIN (feat. 2Rare)
16 Jelly Roll  –  Need A Favor
17 Luke Combs  –  Beautiful Crazy
18 Ghost Riders  –  Happy Now
19 Route 33  –  Simple Things
20 Duncan Toombs  –  A Gift
21 Tyler, The Creator  –  DOGTOOTH
22 Lil Tecca  –  Blessing
23 Trippie Redd  –  1st Degree Murder
24 Mariah the Scientist  –  Only Human
25 Morgan Evans  –  Over For You
26 San Holo  –  DON'T LOOK DOWN (feat. Lizzy Land)
27 San Holo  –  We Rise
28 The Blaze  –  BLOOM
29 The Blaze  –  HAZE
30 boygenius  – 